##### INSTALLED

In [162]:
# pip install --upgrade google-cloud-vision

In [435]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV


# google cloud SDK
import io
import os

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

In [436]:
# Code Reference From: https://github.com/googleapis/google-cloud-python/issues/5349
# need this in order to access API - enter your JSON file in the path after you have created your service account

from google.oauth2 import service_account
credentials = service_account.Credentials. from_service_account_file("./Vision-API/flood-car-classifier-3de49495f20b.json")

# DETECT LABELS
- Function scans photo and returns 10 words to best describe the picture
- I think we should use this first in order to identify water in picture

In [232]:
# formula inspired by https://cloud.google.com/vision/docs/labels

def image_labeler(file_list, string__no_flood_OR_flood):
    
    # Create dictionary for image filename & image labels
    image_dict = {}
    
    for file in file_list:
    
        # Instantiates a client
        client = vision.ImageAnnotatorClient(credentials=credentials)

        # The name of the image file to annotate
        file_name = os.path.abspath('./Vision-API/images/' + string__no_flood_OR_flood + '/' + file)

        # Loads the image into memory
        with io.open(file_name, 'rb') as image_file:
            content = image_file.read()

        image = types.Image(content=content)

        # Performs label detection on the image file
        response = client.label_detection(image=image)
        labels = response.label_annotations
        
        # Create list of image labels
        image_list = []
        
        # Loop through each images assigned labels
        for label in labels:
            image_list.append(label.description)
        
        # Add image filename & labels to dictionary
        image_dict[string__no_flood_OR_flood+file] = image_list
        
    return image_dict

In [233]:
# a list of all of my picture filenames that have floods & cars
flood_list = ['BMW_3series_2.jpeg', 'BMW_3series.jpeg', 'BMW_328i_2.jpg', 'BMW_328i.jpg', 'BMW_330i.jpeg', 'BMW_528i.jpeg', 'BMW_X5.jpeg', 'BMW_X6.jpg', 'BMW.jpeg', 'Ferrari_2.jpeg', 'Ferrari_3.jpeg', 'Ferrari.jpg', 'Lamborghini_waldo.jpeg', 'Lamborghini.jpeg', 'Lamborghini.png', 'Maserati_1.jpeg', 'Maserati_Coupe.jpeg', 'Maserati_Levania.jpeg', 'Maserati_Sedan.jpeg', 'Maserati_SUV_1.jpeg', 'Maserati_SUV.jpeg', 'Mercedes_1.jpeg', 'Mercedes_2.jpeg', 'Mercedes_3.jpg', 'Mercedes_4.jpeg', 'Porsche_911_2.jpeg', 'Porsche_911.jpg', 'RollsRoyce_Ph_2.jpg', 'RollsRoyce_Phantom_2.jpeg', 'RollsRoyce_Phantom.jpeg']

In [234]:
# a list of all of my picture filenames that have cars (no floods)
no_flood_list = ['BMW_328i.jpg', 'BMW_528i_2.jpg', 'BMW_528i.jpg', 'BMW_X3.png', 'BMW_X5.jpg', 'BMW_X6.jpg', 'Ferrari_2.jpeg', 'Ferrari.jpeg', 'Lamborghini_3.jpg', 'Lamborghini_23.jpg', 'Lamborghini.jpg', 'Maserati_2.jpg', 'Maserati_3.jpeg', 'Maserati_Leavnte.jpg', 'Maserati.jpeg', 'Mercedes_CLA.jpg', 'Mercedes_E_class.jpg', 'Mercedes_Sedan.png', 'Mercedes.png', 'Porsche_911_2.jpg', 'Porsche_911_3.jpg', 'Porsche_911_4.png', 'Porsche_911.jpg', 'RollsRoycePhantom.jpg', 'RollsRoycePhantom2.jpg', 'RollsRoycePhantom3.jpg', 'RollsRoycePhantom4.jpg', 'RollsRoycePhantom5.jpg']

In [235]:
# Saving results as variable
flood_dict = image_labeler(flood_list, 'flood')

In [236]:
# Saving results as variable
no_flood_dict = image_labeler(no_flood_list, 'no_flood')

In [437]:
print(no_flood_dict)

{'no_floodBMW_328i.jpg': ['Land vehicle', 'Vehicle', 'Car', 'Motor vehicle', 'Personal luxury car', 'Bmw', 'Performance car', 'Blue', 'Alloy wheel', 'Luxury vehicle'], 'no_floodBMW_528i_2.jpg': ['Land vehicle', 'Vehicle', 'Car', 'Alloy wheel', 'Luxury vehicle', 'Personal luxury car', 'Automotive tire', 'Executive car', 'Tire', 'Spoke'], 'no_floodBMW_528i.jpg': ['Land vehicle', 'Vehicle', 'Car', 'Motor vehicle', 'Executive car', 'Luxury vehicle', 'Personal luxury car', 'Automotive design', 'Bmw', 'Bmw 5 series'], 'no_floodBMW_X3.png': ['Land vehicle', 'Vehicle', 'Car', 'Motor vehicle', 'Bmw', 'Automotive design', 'Personal luxury car', 'Crossover suv', 'Luxury vehicle', 'Bmw x1'], 'no_floodBMW_X5.jpg': ['Land vehicle', 'Vehicle', 'Car', 'Motor vehicle', 'Bmw', 'Spoke', 'Alloy wheel', 'Rim', 'Personal luxury car', 'Bmw x1'], 'no_floodBMW_X6.jpg': ['Land vehicle', 'Vehicle', 'Car', 'Motor vehicle', 'Automotive design', 'Personal luxury car', 'Luxury vehicle', 'Automotive tire', 'Bmw', 'Sp

In [444]:
# create dataframe from results

fl_df = pd.DataFrame(flood_dict.items(), columns=['filename', 'labels'])
fl_df['flood_in_image'] = True

no_fl_df = pd.DataFrame(no_flood_dict.items(), columns=['filename', 'labels'])
no_fl_df['flood_in_image'] = False

noflfl_df = pd.concat([fl_df, no_fl_df], axis=0).reset_index().drop(columns='index')
noflfl_df

,filename,labels,flood_in_image
0,floodBMW_3series_2.jpeg,"[Water, Waterway, Vehicle, Car, Tree, Flood, N...",True
1,floodBMW_3series.jpeg,"[Land vehicle, Vehicle, Car, Road, Driving, Sp...",True
2,floodBMW_328i_2.jpg,"[Vehicle, Car, Night, Automotive lighting, Flo...",True
3,floodBMW_328i.jpg,"[Vehicle, Car, Rain, Mode of transport, Flood,...",True
4,floodBMW_330i.jpeg,"[Motor vehicle, Windshield, Rear-view mirror, ...",True
5,floodBMW_528i.jpeg,"[Vehicle, Car, Flood, Event, River, Watercours...",True
6,floodBMW_X5.jpeg,"[Bridge, Waterway, Vehicle, Water, Car, Water ...",True
7,floodBMW_X6.jpg,"[Land vehicle, Vehicle, Car, Bmw, Mode of tran...",True
8,floodBMW.jpeg,"[Luxury vehicle, Vehicle, Car, Personal luxury...",True
9,floodFerrari_2.jpeg,"[Land vehicle, Vehicle, Car, Supercar, Sports ...",True


In [445]:
def localize_objects_alternate(file_list, string__no_flood_OR_flood):
    
    file_dict = {}
    
    for file in file_list:
    
        client = vision.ImageAnnotatorClient(credentials=credentials)

        with open('./Vision-API/images/' + string__no_flood_OR_flood + '/' + file, 'rb') as image_file:
            content = image_file.read()
        image = vision.types.Image(content=content)

        objects = client.object_localization(image=image).localized_object_annotations
        
        print(file)
        print('Number of objects found: {}'.format(len(objects)))
        
        object_list = []
        
        for object_ in objects:
            object_list.append(object_.name)
            
        file_dict[string__no_flood_OR_flood+file] = object_list
            
    return file_dict

In [446]:
image_objects_flood_alternate = localize_objects_alternate(flood_list, 'flood')

BMW_3series_2.jpeg
Number of objects found: 3
BMW_3series.jpeg
Number of objects found: 3
BMW_328i_2.jpg
Number of objects found: 1
BMW_328i.jpg
Number of objects found: 1
BMW_330i.jpeg
Number of objects found: 2
BMW_528i.jpeg
Number of objects found: 1
BMW_X5.jpeg
Number of objects found: 2
BMW_X6.jpg
Number of objects found: 1
BMW.jpeg
Number of objects found: 8
Ferrari_2.jpeg
Number of objects found: 2
Ferrari_3.jpeg
Number of objects found: 3
Ferrari.jpg
Number of objects found: 1
Lamborghini_waldo.jpeg
Number of objects found: 4
Lamborghini.jpeg
Number of objects found: 2
Lamborghini.png
Number of objects found: 6
Maserati_1.jpeg
Number of objects found: 1
Maserati_Coupe.jpeg
Number of objects found: 4
Maserati_Levania.jpeg
Number of objects found: 2
Maserati_Sedan.jpeg
Number of objects found: 3
Maserati_SUV_1.jpeg
Number of objects found: 2
Maserati_SUV.jpeg
Number of objects found: 1
Mercedes_1.jpeg
Number of objects found: 1
Mercedes_2.jpeg
Number of objects found: 6
Mercedes_

In [307]:
image_objects_flood_alternate

{'floodBMW_3series_2.jpeg': ['Car', 'Van', 'Car'],
 'floodBMW_3series.jpeg': ['Person', 'Car', 'Person'],
 'floodBMW_328i_2.jpg': ['Car'],
 'floodBMW_328i.jpg': ['Car'],
 'floodBMW_330i.jpeg': ['Car', 'Hat'],
 'floodBMW_528i.jpeg': ['Toy vehicle'],
 'floodBMW_X5.jpeg': ['Car', '1D barcode'],
 'floodBMW_X6.jpg': ['Car'],
 'floodBMW.jpeg': ['Car',
  'Person',
  'Car',
  'Pants',
  'Car',
  'Person',
  'Outerwear',
  'Wheel'],
 'floodFerrari_2.jpeg': ['Car', 'Wheel'],
 'floodFerrari_3.jpeg': ['Car', 'Car', 'Wheel'],
 'floodFerrari.jpg': ['Car'],
 'floodLamborghini_waldo.jpeg': ['Car', 'Car', 'Car', 'Truck'],
 'floodLamborghini.jpeg': ['Car', 'Car'],
 'floodLamborghini.png': ['Wheel', 'Car', 'Car', 'Wheel', 'Tire', 'Wheel'],
 'floodMaserati_1.jpeg': ['Car'],
 'floodMaserati_Coupe.jpeg': ['Car', 'Person', 'Person', 'Jeans'],
 'floodMaserati_Levania.jpeg': ['Car', 'Person'],
 'floodMaserati_Sedan.jpeg': ['Car', 'Wheel', 'Car'],
 'floodMaserati_SUV_1.jpeg': ['Car', 'Animal'],
 'floodMaserati_

In [308]:
image_objects_no_flood_alternate = localize_objects_alternate(no_flood_list, 'no_flood')

BMW_328i.jpg
Number of objects found: 5
BMW_528i_2.jpg
Number of objects found: 4
BMW_528i.jpg
Number of objects found: 4
BMW_X3.png
Number of objects found: 4
BMW_X5.jpg
Number of objects found: 4
BMW_X6.jpg
Number of objects found: 7
Ferrari_2.jpeg
Number of objects found: 3
Ferrari.jpeg
Number of objects found: 1
Lamborghini_3.jpg
Number of objects found: 3
Lamborghini_23.jpg
Number of objects found: 3
Lamborghini.jpg
Number of objects found: 3
Maserati_2.jpg
Number of objects found: 6
Maserati_3.jpeg
Number of objects found: 3
Maserati_Leavnte.jpg
Number of objects found: 5
Maserati.jpeg
Number of objects found: 1
Mercedes_CLA.jpg
Number of objects found: 5
Mercedes_E_class.jpg
Number of objects found: 3
Mercedes_Sedan.png
Number of objects found: 4
Mercedes.png
Number of objects found: 4
Porsche_911_2.jpg
Number of objects found: 3
Porsche_911_3.jpg
Number of objects found: 3
Porsche_911_4.png
Number of objects found: 3
Porsche_911.jpg
Number of objects found: 3
RollsRoycePhantom.

In [309]:
image_objects_no_flood_alternate

{'no_floodBMW_328i.jpg': ['Wheel', 'Wheel', 'Wheel', 'Car', 'License plate'],
 'no_floodBMW_528i_2.jpg': ['Wheel', 'Wheel', 'Car', 'Truck'],
 'no_floodBMW_528i.jpg': ['Wheel', 'Wheel', 'Tire', 'Car'],
 'no_floodBMW_X3.png': ['Wheel', 'Wheel', 'Wheel', 'Car'],
 'no_floodBMW_X5.jpg': ['Wheel', 'Wheel', 'Wheel', 'Car'],
 'no_floodBMW_X6.jpg': ['Wheel',
  'Wheel',
  'Wheel',
  'Car',
  'Window',
  'Window',
  'Window'],
 'no_floodFerrari_2.jpeg': ['Wheel', 'Wheel', 'Car'],
 'no_floodFerrari.jpeg': ['Car'],
 'no_floodLamborghini_3.jpg': ['Wheel', 'Car', 'Wheel'],
 'no_floodLamborghini_23.jpg': ['Wheel', 'Wheel', 'Car'],
 'no_floodLamborghini.jpg': ['Wheel', 'Wheel', 'Toy vehicle'],
 'no_floodMaserati_2.jpg': ['Wheel',
  'Wheel',
  'Car',
  'Window',
  'Window',
  'License plate'],
 'no_floodMaserati_3.jpeg': ['Car', 'Wheel', 'Wheel'],
 'no_floodMaserati_Leavnte.jpg': ['Wheel', 'Wheel', 'Car', 'Wheel', 'Window'],
 'no_floodMaserati.jpeg': ['Car'],
 'no_floodMercedes_CLA.jpg': ['Car', 'Wheel'

In [310]:
# create dataframe from results

flood_alt_local = pd.DataFrame(image_objects_flood_alternate.items(), columns=['filename', 'objects'])
flood_alt_local['flood_in_image'] = True

no_flood_alt_local = pd.DataFrame(image_objects_no_flood_alternate.items(), columns=['filename', 'objects'])
no_flood_alt_local['flood_in_image'] = False

aternate_df = pd.concat([flood_alt_local, no_flood_alt_local], axis=0).reset_index().drop(columns='index')
aternate_df

,filename,objects,flood_in_image
0,floodBMW_3series_2.jpeg,"[Car, Van, Car]",True
1,floodBMW_3series.jpeg,"[Person, Car, Person]",True
2,floodBMW_328i_2.jpg,[Car],True
3,floodBMW_328i.jpg,[Car],True
4,floodBMW_330i.jpeg,"[Car, Hat]",True
5,floodBMW_528i.jpeg,[Toy vehicle],True
6,floodBMW_X5.jpeg,"[Car, 1D barcode]",True
7,floodBMW_X6.jpg,[Car],True
8,floodBMW.jpeg,"[Car, Person, Car, Pants, Car, Person, Outerwe...",True
9,floodFerrari_2.jpeg,"[Car, Wheel]",True


In [323]:
aternate_df = aternate_df.sort_values(by='filename')
aternate_df = aternate_df.reset_index().drop(columns='index')
aternate_df = aternate_df.rename(columns={'filename' : 'filename_alt', 'flood_in_image' : 'flood_in_image_alt'})
aternate_df

In [326]:
noflfl_df = noflfl_df.sort_values(by='filename_')
noflfl_df = noflfl_df.reset_index().drop(columns='index')
noflfl_df = noflfl_df.rename(columns={'filename_' : 'filename'})
noflfl_df

,filename,labels,flood_in_image
0,floodBMW.jpeg,"[Luxury vehicle, Vehicle, Car, Personal luxury...",True
1,floodBMW_328i.jpg,"[Vehicle, Car, Rain, Mode of transport, Flood,...",True
2,floodBMW_328i_2.jpg,"[Vehicle, Car, Night, Automotive lighting, Flo...",True
3,floodBMW_330i.jpeg,"[Motor vehicle, Windshield, Rear-view mirror, ...",True
4,floodBMW_3series.jpeg,"[Land vehicle, Vehicle, Car, Road, Driving, Sp...",True
5,floodBMW_3series_2.jpeg,"[Water, Waterway, Vehicle, Car, Tree, Flood, N...",True
6,floodBMW_528i.jpeg,"[Vehicle, Car, Flood, Event, River, Watercours...",True
7,floodBMW_X5.jpeg,"[Bridge, Waterway, Vehicle, Water, Car, Water ...",True
8,floodBMW_X6.jpg,"[Land vehicle, Vehicle, Car, Bmw, Mode of tran...",True
9,floodFerrari.jpg,"[Land vehicle, Vehicle, Car, Automotive design...",True


In [374]:
df_all = pd.concat([noflfl_df, aternate_df], axis=1).drop(columns=['filename_alt', 'flood_in_image_alt'])
df_all.head()

,filename,labels,flood_in_image,objects
0,floodBMW.jpeg,"[Luxury vehicle, Vehicle, Car, Personal luxury...",True,"[Car, Person, Car, Pants, Car, Person, Outerwe..."
1,floodBMW_328i.jpg,"[Vehicle, Car, Rain, Mode of transport, Flood,...",True,[Car]
2,floodBMW_328i_2.jpg,"[Vehicle, Car, Night, Automotive lighting, Flo...",True,[Car]
3,floodBMW_330i.jpeg,"[Motor vehicle, Windshield, Rear-view mirror, ...",True,"[Car, Hat]"
4,floodBMW_3series.jpeg,"[Land vehicle, Vehicle, Car, Road, Driving, Sp...",True,"[Person, Car, Person]"


In [448]:
df_all.flood_in_image = [1 if row == True else 0 for row in df_all.flood_in_image]

In [449]:
df_all['labels_and_objects'] = df_all.labels + df_all.objects

In [450]:
df_all['labels_and_objects'] = df_all['labels_and_objects'].apply(lambda x: " ".join(x).lower())

In [451]:
df_all.head()

,filename,labels,flood_in_image,objects,labels_and_objects
0,floodBMW.jpeg,"[Luxury vehicle, Vehicle, Car, Personal luxury...",1,"[Car, Person, Car, Pants, Car, Person, Outerwe...",luxury vehicle vehicle car personal luxury car...
1,floodBMW_328i.jpg,"[Vehicle, Car, Rain, Mode of transport, Flood,...",1,[Car],vehicle car rain mode of transport flood event...
2,floodBMW_328i_2.jpg,"[Vehicle, Car, Night, Automotive lighting, Flo...",1,[Car],vehicle car night automotive lighting flood ev...
3,floodBMW_330i.jpeg,"[Motor vehicle, Windshield, Rear-view mirror, ...",1,"[Car, Hat]",motor vehicle windshield rear-view mirror auto...
4,floodBMW_3series.jpeg,"[Land vehicle, Vehicle, Car, Road, Driving, Sp...",1,"[Person, Car, Person]",land vehicle vehicle car road driving sports s...


In [452]:
df_all.to_csv('./data/label_multobj.csv')

In [429]:
X = df_all.labels_and_objects
y = df_all.flood_in_image

In [430]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=2020)

In [431]:
cvec = CountVectorizer()
Z_train = cvec.fit_transform(X_train)
Z_test = cvec.transform(X_test)

In [443]:
cross_val_score(LogisticRegression(), Z_train, y_train, cv=5).mean()

0.8305555555555555

In [432]:
logreg = LogisticRegression()
logreg.fit(Z_train, y_train)

LogisticRegression()

In [433]:
print(f'Training Score: {logreg.score(Z_train, y_train)}')
print(f'Testing Score: {logreg.score(Z_test, y_test)}')

Training Score: 0.9767441860465116
Testing Score: 0.8666666666666667


In [434]:
# Baseline Score
df_all.flood_in_image.value_counts(normalize=True)

1    0.517241
0    0.482759
Name: flood_in_image, dtype: float64

# OLDER ISH

In [239]:
# # Create alternate Dataframe from results

# flood_df = pd.DataFrame(
#     [(k, v) for (k, L) in flood_dict.items() for v in L], 
#     columns=['image_file_name', 'labels']
# )
# flood_df['flood_in_image'] = 'yes'

# flood_df['image_file_name'] = flood_df['flood_in_image'] + '_' + flood_df['image_file_name']
# flood_df.head()

In [240]:
# # Create alternate Dataframe from results

# no_flood_df = pd.DataFrame(
#     [(k, v) for (k, L) in no_flood_dict.items() for v in L], 
#     columns=['image_file_name', 'labels']
# )
# no_flood_df['flood_in_image'] = 'no'

# no_flood_df['image_file_name'] = no_flood_df['flood_in_image'] + '_' + no_flood_df['image_file_name']
# no_flood_df.head()

In [241]:
# # concatenate to make alternate df
# sample_df = pd.concat([flood_df, no_flood_df])
# sample_df

In [242]:
# # Groupby
# sample_df.groupby('labels')[['image_file_name']].count().sort_values(by='image_file_name', ascending=False)

In [243]:
# sorted(sample_df.labels.unique())

# # Words that may suggest flooding pulled from above and manually added to list
# flood_key_words = ['Geological phenomenon', 'Low water crossing', 'Mud', 'Precipitation', 'Rain', 'Rescue', 'River', 'Stream', 'Water', 'Water resources', 'Watercourse', 'Waterway',]

In [244]:
# modeling_df = sample_df
# modeling_df

In [245]:
# modeling_df['flood_in_image'] = [1 if i == 'yes' else 0 for i in modeling_df['flood_in_image']]
# modeling_df

In [246]:
# modeling_df = pd.get_dummies(modeling_df, columns=['labels'], drop_first=True)
# modeling_df

In [247]:
# modeling_df = modeling_df.groupby('image_file_name').sum()
# modeling_df.head()

In [248]:
# modeling_df['flood_in_image'] = [1 if i > 0 else 0 for i in modeling_df['flood_in_image']]

In [249]:
# modeling_df.shape

In [255]:
# # Baseline
# modeling_df.flood_in_image.value_counts(normalize=True)

In [256]:
# X = modeling_df.drop(columns=['flood_in_image'])
# y = modeling_df.flood_in_image

In [257]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=2020)

In [258]:
# lr = LinearRegression()
# lr.fit(X_train, y_train)

In [259]:
# print(f'Training Score: {lr.score(X_train, y_train)}')
# print(f'Testing Score: {lr.score(X_test, y_test)}')

In [266]:
def localize_objects_tony(file_list, string__no_flood_OR_flood):
    
    file_dict = {}
    
    for file in file_list:
    
        client = vision.ImageAnnotatorClient(credentials=credentials)

        with open('./Vision-API/images/' + string__no_flood_OR_flood + '/' + file, 'rb') as image_file:
            content = image_file.read()
        image = vision.types.Image(content=content)

        objects = client.object_localization(image=image).localized_object_annotations
        
        print(file)
        print('Number of objects found: {}'.format(len(objects)))
        
        object_dict = {}
        
        for index, object_ in enumerate(objects):
            object_dict['object'+str(index+1)+'_name'] = object_.name
            object_dict['object'+str(index+1)+'_score'] = object_.score
            
        file_dict[string__no_flood_OR_flood+file] = object_dict
            
    return file_dict

In [300]:
# image_objects_flood = localize_objects_tony(flood_list, 'flood')

In [301]:
# image_objects_flood

In [302]:
# image_objects_no_flood = localize_objects_tony(no_flood_list, 'no_flood')

In [303]:
# image_objects_no_flood

In [311]:
# localize_flood_df = pd.DataFrame(image_objects_flood).T
# localize_flood_df.head()

In [312]:
# localize_no_flood_df = pd.DataFrame(image_objects_no_flood).T
# localize_no_flood_df.head()

In [313]:
# localize_df = pd.concat([localize_no_flood_df, localize_flood_df], axis=0).reset_index().rename(columns={'index' : 'filename'})
# localize_df

In [314]:
# noflfl_df.shape

In [315]:
# localize_df.shape

In [316]:
# localize_df = localize_df.sort_values(by='filename')
# localize_df = localize_df.reset_index().drop(columns='index')
# localize_df

### ALL FUNCTIONS BELOW TAKEN FROM GOOGLE

# COLOR / PROPERTIES detector

In [260]:
def detect_properties(path):
    """Detects image properties in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient(credentials=credentials)

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.image_properties(image=image)
    props = response.image_properties_annotation
    print('Properties:')

    for color in props.dominant_colors.colors:
        print('fraction: {}'.format(color.pixel_fraction))
        print('\tr: {}'.format(color.color.red))
        print('\tg: {}'.format(color.color.green))
        print('\tb: {}'.format(color.color.blue))
        print('\ta: {}'.format(color.color.alpha))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [261]:
# detect_properties('./Vision-API/images/flood/BMW_528i.jpeg')

# LOGO detector

In [262]:
def detect_logos(path):
    """Detects logos in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient(credentials=credentials)

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.logo_detection(image=image)
    logos = response.logo_annotations
    print('Logos:')

    for logo in logos:
        print(logo.description)

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [263]:
# detect_logos('./Vision-API/images/flood/BMW_528i.jpeg')

# LABEL detector

In [264]:
# def detect_labels(path):
#     """Detects labels in the file."""
#     from google.cloud import vision
#     import io
#     client = vision.ImageAnnotatorClient(credentials=credentials)

#     with io.open(path, 'rb') as image_file:
#         content = image_file.read()

#     image = vision.types.Image(content=content)

#     response = client.label_detection(image=image)
#     labels = response.label_annotations
#     print('Labels:')

#     for label in labels:
#         print(label.description)

#     if response.error.message:
#         raise Exception(
#             '{}\nFor more info on error messages, check: '
#             'https://cloud.google.com/apis/design/errors'.format(
#                 response.error.message))

# MULTIPLE OBJECT detector

In [265]:
# def localize_objects(path):
#     """Localize objects in the local image.

#     Args:
#     path: The path to the local file.
#     """
#     from google.cloud import vision
#     client = vision.ImageAnnotatorClient(credentials=credentials)

#     with open(path, 'rb') as image_file:
#         content = image_file.read()
#     image = vision.types.Image(content=content)

#     objects = client.object_localization(image=image).localized_object_annotations

#     print('Number of objects found: {}'.format(len(objects)))
#     for object_ in objects:
#         print('\n{} (confidence: {})'.format(object_.name, object_.score))
#         print('Normalized bounding polygon vertices: ')
#         for vertex in object_.bounding_poly.normalized_vertices:
#             print(' - ({}, {})'.format(vertex.x, vertex.y))